# IMPORT

In [42]:
from __future__ import annotations
from bqskit import Circuit, compile
from bqskit.compiler.machine import MachineModel
from bqskit.ir.gates import RXXGate, RXGate, RZGate,RYGate,RZZGate,HGate,CNOTGate
from transpiler.BQSkit_custom_gate_definitions import VirtualZGate, GPIGate, GPI2Gate,PartialMSGate,FullMSGate
from transpiler.QASM2_reader import load_qasm
from bqskit.ext import bqskit_to_qiskit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit import transpile
from qiskit.transpiler import Target, InstructionProperties
import pennylane as qml
import numpy as np

In [2]:
# Step 1: Define the MS-Gate Machine Model
IONQ_gate_set={PartialMSGate(),GPIGate(),GPI2Gate()}
def transpile_to_ms(circuit: Circuit) -> Circuit:
    """Transpile the given circuit to use only MS gates."""
    model = MachineModel(circuit.num_qudits, gate_set=IONQ_gate_set)
    compiled_circuit = compile(circuit, model, optimization_level=3)
    return compiled_circuit

# Step 2: Create a simple quantum circuit
qc = Circuit(2)
qc.append_gate(HGate(), [0])
qc.append_gate(RZZGate(), [0, 1],params=[1])
qc.append_gate(RYGate(), [1],params=[3])
qc.append_gate(RXXGate(), [0,1],params=[2.14584545])
qc.append_gate(RZZGate(), [1,0],params=[1.1561])
qc.append_gate(RXGate(), [1],params=[1.1561])
qc_qiskit = bqskit_to_qiskit(qc)

# Step 3: Transpile the circuit to MS gates
ms_circuit = transpile_to_ms(qc)

In [3]:
#SAVE QASM
ms_circuit.save("circuit.qasm")
print(qc_qiskit.draw(output='text'))

     ┌───┐                 ┌──────────────┐                           
q_0: ┤ H ├─■───────────────┤0             ├─■─────────────────────────
     └───┘ │ZZ(1) ┌───────┐│  Rxx(2.1458) │ │ZZ(1.1561) ┌────────────┐
q_1: ──────■──────┤ Ry(3) ├┤1             ├─■───────────┤ Rx(1.1561) ├
                  └───────┘└──────────────┘             └────────────┘


In [4]:
# Step 4: Transform to qiskit and print
qiskit_circuit = load_qasm("circuit.qasm")
print(qiskit_circuit.draw(output='text'))


     ┌────────────────────────────┐┌─────────┐»
q_0: ┤0                           ├┤ Gpi2(1) ├»
     │  Partialms(4.5,0.5,3.8047) │├─────────┤»
q_1: ┤1                           ├┤ Gpi2(6) ├»
     └────────────────────────────┘└─────────┘»
«     ┌─────────────────────────────────┐┌─────────────────┐ ┌──────────────┐»
«q_0: ┤0                                ├┤ Gpi2(0.0097291) ├─┤ Gpi2(2.0097) ├»
«     │  Partialms(5.8334,3.027,5.7105) │└─┬─────────────┬─┘┌┴──────────────┤»
«q_1: ┤1                                ├──┤ Gpi2(5.777) ├──┤ Gpi2(0.71098) ├»
«     └─────────────────────────────────┘  └─────────────┘  └───────────────┘»
«      ┌───────────┐ 
«q_0: ─┤ Gpi2(2.5) ├─
«     ┌┴───────────┴┐
«q_1: ┤ Gpi(2.9805) ├
«     └─────────────┘


# NOISE

In [43]:
# Convert Qiskit circuit to PennyLane
qml_circuit = qml.from_qiskit(qiskit_circuit)
np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2#check

array([0.54795677, 0.00186807, 0.43902641, 0.01114875])

In [47]:
@qml.qnode(qml.device("default.mixed", wires=2))  
def noisy_circuit():
    tape = qml.transforms.make_tape(qml_circuit)()

    for op in tape.operations:

        qml.QubitUnitary(op.parameters[0], wires=op.wires)  

        if len(op.wires) == 2:  # MS gates
            for qubit in op.wires:  
                qml.DepolarizingChannel(0.015, wires=qubit)  # Adjusted probability

    return qml.probs(wires=[0, 1])

noisy_result = noisy_circuit()
print("Noisy simulation result:", noisy_result)


хуй [array([[ 8.17613940e-01+0.j        ,  0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        , -1.70948868e-07+0.57576683j],
       [ 0.00000000e+00+0.j        ,  8.17613940e-01+0.j        ,
         1.00757224e-08+0.57576683j,  0.00000000e+00+0.j        ],
       [ 0.00000000e+00+0.j        , -1.00757224e-08+0.57576683j,
         8.17613940e-01+0.j        ,  0.00000000e+00+0.j        ],
       [ 1.70948868e-07+0.57576683j,  0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        ,  8.17613940e-01+0.j        ]])]
хуй [array([[ 7.07106781e-01+0.j        ,  8.56079619e-08-0.70710678j],
       [-8.56079619e-08-0.70710678j,  7.07106781e-01+0.j        ]])]
хуй [array([[ 7.07106781e-01+0.j        ,  1.38494244e-07-0.70710678j],
       [-1.38494244e-07-0.70710678j,  7.07106781e-01+0.j        ]])]
хуй [array([[ 0.61409513+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.60680425+0.50465411j],
       [ 0.        +0.j        ,  0.61409513+0.j        ,

In [38]:
qiskit_circuit.num_qubits

2

In [37]:


# Создаем шумовую модель
noise_model = NoiseModel()

# 3% деполяризационная ошибка для PartialMS
partialMS_noise = depolarizing_error(1, 2)

# Добавляем базовые гейты (ВАЖНО: unitary fallback)
noise_model.add_basis_gates(["unitary"])
noise_model.add_all_qubit_quantum_error(partialMS_noise, ["partialMS"])

# Создаем Target с явным указанием кубитов
target = Target(num_qubits=2)

# Добавляем 1-кубитные гейты
target.add_instruction(GPIGate(0), {(0,): InstructionProperties(duration=1.0), (1,): InstructionProperties(duration=1.0)})
target.add_instruction(GPI2Gate(0), {(0,): InstructionProperties(duration=1.0), (1,): InstructionProperties(duration=1.0)})
target.add_instruction(VirtualZGate(0), {(0,): InstructionProperties(duration=0.5), (1,): InstructionProperties(duration=0.5)})

# Добавляем 2-кубитные гейты
target.add_instruction(PartialMSGate(0, 0, 0), {(0, 1): InstructionProperties(duration=2.0)})
target.add_instruction(FullMSGate(0, 0), {(0, 1): InstructionProperties(duration=2.0)})

# Создаем симулятор с кастомными гейтами и unitary fallback
simulator = AerSimulator(
    noise_model=noise_model,
    target=target,
    basis_gates=["unitary", "partialMS", "gpi", "gpi2"],  # Добавляем unitary
    custom_unitary={PartialMSGate()}  # Включаем поддержку произвольных унитарных операций
)

# Транспиляция схемы
tqc = transpile(qiskit_circuit, simulator, basis_gates=["unitary", "partialMS", "gpi", "gpi2"])

print(tqc.draw(output='text'))

# Запуск симуляции
result = simulator.run(tqc).result()

# Вывод результатов
print(result.get_counts())


AerError: 'Invalid option custom_unitary'

In [29]:




# Проверяем, какие гейты зарегистрированы
print("Registered instructions in target:", list(target.keys()))


Registered instructions in target: ['GPIGate', 'GPI2Gate', 'VirtualZGate', 'PartialMSGate', 'FullMSGate']
